# Notebook match titres avec Garnier
Ce notebook permet de faire correspondre les titres d'oeuvres stockés dans un tableau et le fichier du Garnier.

**Fonctionnalités présentes :**
- récupère le thesaurus en format json
- parcourt sa hierarchie
- effectue une correspondance exacte du titre avec le thesaurus
  - si aucun résultat, supprime les déterminants
  - effectue une recherche du mot au sein du thésaurus


**Limites :**
- ne permet pas d'associer des termes, impliquer des sous-entendus (Actéon avec les Métamorphoses)
- faux-positifs : associe des termes correspondant exactement mais qui n'ont cependant pas de lien (le signe du zodiaque vierge avec la vierge marie)



**Fonctionnalités à ajouter**
- utilisation d'un fichier organisé comme une liste
- une gestion de conflit :
  - si plusieurs termes apparaissent et ne correspondent pas à la même catégorie principale, une indication est soulevée

- une amélioration des sous-entendus :
  - Annonciation implique la vierge et l'ange gabriel
  - Sainte trinité implique dieu le père, colombe du saint esprit,...

# Étape 1 : préparation des données

- récupération du fichier json avec les termes du Thésaurus Garnier au format json
- préparation des termes dans un tableau

In [ ]:
# récupération des données json
import os
import json

thesaurus_json = "JSONthesaurusGarnierINHA.json"

base_donnee = "4" # numéro de la base de donnée dans AGORHA

# ouverture du Garnier en fichier json, stockage dans json_load
with open(thesaurus_json, "r", encoding="utf-8") as f:
    json_load = json.load(f)

# Etape 2 : Requête des notices et des titres depuis AGORHA

In [ ]:
import os
import json
import requests

# === Création du json contenant toutes les notices pour la base de données ===

dir_export_json = "export_json"
if not os.path.exists(dir_export_json):
    os.makedirs(dir_export_json)

json_API = os.path.join(dir_export_json, f"export_{base_donnee}.json")

def export_db_json(num, jeton = None, all_notices=None) :
    if all_notices is None:
        all_notices = []

    params = {"noticeType": "ARTWORK", # à modifier en fonction du type de données ("ARTWORK")
              "database": str(base_donnee), # à modifier en fonction de la base de données
              "token": jeton}

    requete = requests.get('https://agorha.inha.fr/api/notice/exportjson', params=params)
    # print(requete.url)

    reponse = requete.json()
    jeton = reponse.get("token", "")
    # print(jeton)
    notices = reponse.get("notices", [])
    # print(notices)
    all_notices.extend(notices)

    # les lignes ci-dessous sont à activer pour avoir les json par pages
    #notice_ouput = os.path.join(dir_export_json, f"notices_{num}.json")
    #with open(notice_ouput, "w", encoding="utf-8") as f:
    #    json.dump(notices, f, ensure_ascii=False)

    if len(jeton):
            export_db_json(str((int(num) + 1)), jeton=jeton, all_notices = all_notices)
    else :
        with open(json_API, "w", encoding="utf-8") as f_all:
            json.dump({"notices":all_notices}, f_all, ensure_ascii=False)
        print(f"Les notices ont été exportés dans le fichier {json_API}")

resultat = export_db_json("1")

Les notices ont été exportés dans le fichier export_json/export_4.json


In [ ]:
""" fonction pour récupérer les titres depuis l'export de l'API """

import os
import json
import re

dir_export_json = "export_json"
json_API = os.path.join(dir_export_json, f"export_{base_donnee}.json")

dir_titres_json = "json_titres"
os.makedirs(dir_titres_json, exist_ok=True)
json_export_titres = os.path.join(dir_titres_json, f"export_titres_{base_donnee}.json")

# Charger le JSON source
with open(json_API, "r", encoding="utf-8") as f:
    data_raw = json.load(f)

# Extraire les notices
rows = []
for notice in data_raw.get("notices", []):
    title = (
        notice.get("content", {})
        .get("identificationInformation", {})
        .get("title", [{}])[0]
        .get("label", {})
        .get("value", "")
    )
    uri = notice.get("internal", {}).get("permalink", "")

    # Supprimer toutes les balises HTML <...>
    title_clean = re.sub(r"<.*?>", "", title).strip()

    rows.append({"uri": uri, "title": title_clean})

# Sauvegarder en JSON
with open(json_export_titres, "w", encoding="utf-8") as f:
    json.dump(rows, f, ensure_ascii=False, indent=2)

print(f"JSON créé : {json_export_titres} avec {len(rows)} notices")


JSON créé : json_titres/export_titres_4.json avec 14062 notices


# Etape 3 : fonctions de recherche


In [ ]:
import re

"""
Fonction de nettoyage des titres
- supprime les articles
- supprime les majuscules
"""
def nettoyage_titre(titre):
    # Liste des articles à supprimer
    articles = [
        r"\bl'", r"\ble ", r"\bla ", r"\bles ", r"\bun ", r"\bune ", r"\bdes ", r"\bd'",
        r"\bdu ", r"\bde la ", r"\bde l'", r"\baux ", r"\bau ", r"\bde ", r"\ben ",
        r"\bà ", r"\bdans ", r"\bpar ", r"\bpour ", r"\bvers "
    ]
    titre = titre.lower()
    resultat = titre

    # Remplacer tous les articles par une chaîne vide
    for article in articles:
        resultat = re.sub(article, '', resultat)

    # nettoie les espaces supplémentaires
    resultat = re.sub(r'\s+', ' ', resultat).strip()

    return resultat

In [ ]:
def chercher_saint(json, titre, parents=[]):
    # Déclenche uniquement si "saint" ou "sainte" est dans le titre
    # cette fonction est à améliorer, elle ne fonctionne pas bien
    if not ("saint" in titre or "sainte" in titre):
        return []

    saints_nodes = trouver_branche_saints(json)
    resultats = []

    if saints_nodes:
        for node in saints_nodes:
            resultats.extend(chercher_label_partiel([node], titre, parents=[]))

    return resultats

def trouver_branche_saints(json):
    for concept in json:
        if concept["label"].lower() == "personnage historique":
            children = concept.get("children", [])
            return [
                child for child in children
                if "saint identifié" in child["label"].lower()
                or "sainte identifiée" in child["label"].lower()
            ]
    return []

In [ ]:
"""
Fonction de recherche exacte d'un label dans le titre
- recherche d'une correspondance EXACTE entre le titre et un concept Garnier
- cherche les parents du concept trouvé
"""
def chercher_label(json, titre, parents=[]):
    for concept in json:
        if concept["label"].lower() == titre.lower():
            # cherche une correspondance exacte d'un label dans un titre
            return concept, parents  # Retourne l'élément trouvé et tous ses parents
        if "children" in concept:
            # Ajouter le parent actuel à la liste des parents et continuer la recherche
            resultat, parents_trouve = chercher_label(concept["children"], titre, parents + [concept])
            if resultat:
                return resultat, parents_trouve
    return None, None  # Si rien trouvé

In [ ]:
"""
Fonction de recherche partielle d'un label dans le titre
- recherche d'un concept Garnier dont le label est présent partiellement
dans le titre du tableau
- cherche les parents du concept trouvé
"""
def chercher_label_partiel(json, titre, parents=[]):
    resultats = []
    for concept in json:
        if titre.lower() in concept["label"].lower():
            resultats.append((concept, parents))
        label = concept["label"].lower()

        # pattern récupère le mot complet du label (pas une partie) et cherche les pluriels
        pattern = r'\b' + re.escape(label) + r'\b'
        pattern_pl_s = r'\b' + re.escape(label) + r's\b'
        pattern_pl_x = r'\b' + re.escape(label) + r'x\b'
        #pattern_fem_sg = r'\b' + re.escape(label) + r'e\b'
        #pattern_fem_pl = r'\b' + re.escape(label) + r'es\b'

        if label.endswith('al'):
            label_aux = re.sub(r'(al)$', 'aux', label)
            pattern_pl_aux = r'\b' + re.escape(label_aux) + r'\b'
        else : pattern_pl_aux = None

        if (
            re.search(pattern, titre.lower())
            or re.search(pattern_pl_s, titre.lower())
            or re.search(pattern_pl_x, titre.lower())
            or (pattern_pl_aux and re.search(pattern_pl_aux, titre_lower))
            ) :
            resultats.append((concept, parents))

        if "children" in concept:
            resultats_enfants = chercher_label_partiel(concept["children"], titre, parents + [concept])
            resultats.extend(resultats_enfants)
    return resultats

In [ ]:
import re

def nettoyage_garnier(json):
    """
    Nettoie les labels dans la structure JSON :
    - met en minuscules
    - supprime les textes entre parenthèses
    """
    for concept in json:
        label = concept["label"].lower()
        label = re.sub(r'\s*\(.*?\)', '', label).strip()  # Supprime parenthèses et espace avant/après
        concept["label"] = label

        if "children" in concept:
            nettoyage_garnier(concept["children"])

In [ ]:
import os
import json

# fichiers d'entrée
dir_titres_json = "json_titres"
json_export_titres = os.path.join(dir_titres_json, f"export_titres_{base_donnee}.json")

with open(json_export_titres, "r", encoding="utf-8") as f:
    data = json.load(f)
total = len(data)
compteur = 0

# Fichier de sortie
dir_export_labels = "export_labels_titres"
os.makedirs(dir_export_labels, exist_ok=True)
json_resultats_path = os.path.join(dir_export_labels, f"resultats_garnier_titres_{base_donnee}.json")

# Charger résultats existants s’il y en a déjà
if os.path.exists(json_resultats_path):
    with open(json_resultats_path, "r", encoding="utf-8") as f:
        resultats_json = json.load(f)
else:
    resultats_json = []

# Pour éviter les doublons si on relance le script
uris_deja_traites = {item["uri"] for item in resultats_json}

# Parcourir les titres
for item in data:
    uri = item["uri"]
    titre = item["title"]
    compteur += 1

    if uri in uris_deja_traites:
        print(f"{compteur}/{total} passé : {titre}, déjà traitée")
        continue  # on passe si déjà enregistré

    titre_lower = titre.lower()
    termes_trouves = []

    # début fonction chercher-saint
    if "saint" in titre_lower or "sainte" in titre_lower:
        resultats_saints = chercher_saint(json_load, titre_lower)
        if resultats_saints:
            for match, parents in resultats_saints:
                termes_trouves.append(match["label"])
            # Sauvegarde immédiate dans le JSON
            resultats_json.append({"uri": uri, "title": titre, "termes": termes_trouves})
            with open(json_resultats_path, "w", encoding="utf-8") as f:
                json.dump(resultats_json, f, ensure_ascii=False, indent=2)
            continue

    match, parents = chercher_label(json_load, titre_lower)
    if not match:
        titre_clean = nettoyage_titre(titre_lower)
        nettoyage_garnier(json_load)
        match, parents = chercher_label(json_load, titre_clean)

    if match:
        termes_trouves.append(match["label"])

    else:
        resultats_partiels = chercher_label_partiel(json_load, titre_lower)
        if not resultats_partiels:
            titre_clean = nettoyage_titre(titre_lower)
            resultats_partiels = chercher_label_partiel(json_load, titre_clean)

        if resultats_partiels:
            for match, parents in resultats_partiels:
                termes_trouves.append(match["label"])

    # Ajout et sauvegarde progressive
    resultats_json.append({"uri": uri, "title": titre, "termes": termes_trouves})
    with open(json_resultats_path, "w", encoding="utf-8") as f:
        json.dump(resultats_json, f, ensure_ascii=False, indent=2)

    print(f"{compteur}/{total} traité : {titre}")

Streaming output truncated to the last 5000 lines.
8899/14062 passé : Port de mer, déjà traitée
8900/14062 passé : Vue d'un canal bordé d'architectures, déjà traitée
8901/14062 passé : Ruines romaines, déjà traitée
8902/14062 passé : Nature morte à la raie, aux crabes et coquillages, déjà traitée
8903/14062 passé : Saint Nicolas sauvant des naufragés, déjà traitée
8904/14062 passé : Oiseaux et coquillages, déjà traitée
8905/14062 passé : Bataille, déjà traitée
8906/14062 passé : Nature morte aux raisins, pastèque, melon et autres fruits, déjà traitée
8907/14062 passé : Autoportrait, déjà traitée
8908/14062 passé : Réjouissance campagnarde devant un château, déjà traitée
8909/14062 passé : Pietà, déjà traitée
8910/14062 passé : Saint Jean l'Evangéliste, déjà traitée
8911/14062 passé : Portrait de femme, déjà traitée
8912/14062 passé : La Crucifixion, déjà traitée
8913/14062 passé : La Sainte Famille, déjà traitée
8914/14062 passé : David obtient la cessation de la peste, déjà traitée
89

In [ ]:
import csv
from collections import Counter

def exporter_termes_csv(termes, chemin_csv='resultats_termes.csv'):
    """
    Exporte une liste de termes dans un fichier CSV avec leur nombre d'occurrences.
    Format : terme;occurence
    """
    compteur = Counter(termes)
    with open(chemin_csv, 'w', newline='', encoding='utf-8') as fichier_csv:
        writer = csv.writer(fichier_csv, delimiter=';')
        writer.writerow(['terme', 'occurence'])
        for terme, freq in compteur.most_common():
            writer.writerow([terme, freq])

exporter_termes_csv(termes_trouves)

# Proposition d'erreurs

# Création de liens sous-entendus



## utilisation d'un dataset déjà repertorié
